In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
import logging

# Start Spark
spark = SparkSession.builder \
    .appName("BTC_Prediction") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")  
logging.getLogger("py4j").setLevel(logging.ERROR)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/21 14:21:22 WARN Utils: Your hostname, chaima-ThinkPad-T470-W10DG, resolves to a loopback address: 127.0.1.1; using 192.168.1.120 instead (on interface wlp4s0)
26/01/21 14:21:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/21 14:21:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Load silver data
silver_path = "../data/silver/silver_dataset"
df = spark.read.parquet(silver_path)
df.show(5)

+-------------------+--------+--------+--------+--------+-------+--------------------+------------------+----------------+---------------------+----------------------+------+---------------+--------------------+-----------------+-----------------+-------------------+
|          open_time|    open|    high|     low|   close| volume|          close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|close_t_plus_10|           return_1m|             MA_5|            MA_10|        taker_ratio|
+-------------------+--------+--------+--------+--------+-------+--------------------+------------------+----------------+---------------------+----------------------+------+---------------+--------------------+-----------------+-----------------+-------------------+
|2026-01-19 09:53:00|93046.35|93046.35|92997.65| 93001.8|9.10532|2026-01-19 09:53:...|    847027.2616431|            2024|              2.08878|        194272.7849138|     0|       92965.51|      

In [3]:
median_value=df.approxQuantile("close_t_plus_10", [0.5], 0.1)[0]
print("median of target:", median_value)

median of target: 93044.38


In [4]:
# identify numeric columns
numeric_cols = [
    c for c, t in df.dtypes
    if t in ["int", "bigint", "double", "float"]
    and c != "close_t_plus_10"
]

In [5]:
#assemble features
assembler = VectorAssembler(
    inputCols=numeric_cols,
    outputCol="features"
)

In [6]:
df_ml = assembler.transform(df).select(
    "open_time",
    "features",
    "close_t_plus_10"
)

In [7]:
# count rows
total = df_ml.count()
train_limit = int(total * 0.8)

In [8]:
# index by time
window = Window.orderBy("open_time")
df_ml = df_ml.withColumn("rn", F.row_number().over(window))

In [9]:
# split
train_data = df_ml.filter(F.col("rn") <= train_limit) \
                  .drop("rn", "open_time")

test_data = df_ml.filter(F.col("rn") > train_limit) \
                 .drop("rn", "open_time")

In [10]:

lr = LinearRegression(featuresCol="features", labelCol="close_t_plus_10")
model = lr.fit(train_data)
predictions = model.transform(test_data)



26/01/21 14:22:04 ERROR Executor: Exception in task 0.0 in stage 8.0 (TID 6)/ 1]
org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] User defined function (`VectorAssembler$$Lambda/0x00007a21591dc850`: (struct<open:double,high:double,low:double,close:double,volume:double,quote_asset_volume:double,number_of_trades_double_VectorAssembler_0623ff60dc60:double,taker_buy_base_volume:double,taker_buy_quote_volume:double,return_1m:double,MA_5:double,MA_10:double,taker_ratio:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) failed due to: org.apache.spark.SparkException: Encountered null while assembling a row with handleInvalid = "error". Consider
removing nulls from dataset or using handleInvalid = "keep" or "skip".. SQLSTATE: 39000
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:197)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecution

Py4JJavaError: An error occurred while calling o94.fit.
: org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] User defined function (`VectorAssembler$$Lambda/0x00007a21591dc850`: (struct<open:double,high:double,low:double,close:double,volume:double,quote_asset_volume:double,number_of_trades_double_VectorAssembler_0623ff60dc60:double,taker_buy_base_volume:double,taker_buy_quote_volume:double,return_1m:double,MA_5:double,MA_10:double,taker_ratio:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) failed due to: org.apache.spark.SparkException: Encountered null while assembling a row with handleInvalid = "error". Consider
removing nulls from dataset or using handleInvalid = "keep" or "skip".. SQLSTATE: 39000
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:197)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.a

In [ ]:
#evaluate model
rmse = RegressionEvaluator(labelCol="close_t_plus_10", predictionCol="prediction", metricName="rmse")
mae = RegressionEvaluator(labelCol="close_t_plus_10",predictionCol="prediction",metricName="mae")

rmse = rmse.evaluate(predictions)
mae = mae.evaluate(predictions)

In [ ]:
#printing results
print(f"root mean square error (RMSE): {rmse}")
print(f"mean absolute error (MAE): {mae}")


root mean square error (RMSE): 62.80099893403466
mean absolute error (MAE): 50.74993129601723
